In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Part 1

In [3]:
'R 6 (#70c710)'

'R 6 (#70c710)'

In [4]:
int("70c71", 16)


461937

In [28]:
import numpy as np

class Edge:
    def __init__(self, cur_loc, direction, steps, color):
        self.color = color.replace(')','').replace('(','')
        changes = {
            'U' : np.array([-1,0]),
            'D' : np.array([1,0]),
            'R' : np.array([0,1]),
            'L' : np.array([0,-1]),
        }
        self.start = cur_loc
        self.end = np.array(cur_loc) +(changes[direction] * int(steps))
        
        self.direction = direction
        self.steps = int(steps)
        
    def __repr__(self):
        return f'{self.start} -> {self.end}'

In [29]:
from collections import deque

def find_dimensions(edges):
    x_range = [0,0]
    y_range = [0,0]
    
    for e in edges:
        for loc in [e.start, e.end]:
            if loc[0] < y_range[0] :
                y_range[0] = loc[0]
            if loc[0] > y_range[1] :
                y_range[1] = loc[0]
            if loc[1] < x_range[0] :
                x_range[0] = loc[1]
            if loc[1] > x_range[1] :
                x_range[1] = loc[1]
    return x_range, y_range

def create_grid(edges):
    xr,yr = find_dimensions(edges)
    
    grid = np.zeros( (yr[1]-yr[0]+3,xr[1]-xr[0]+3))
    
    yshift = -1*yr[0]+1
    xshift = -1*xr[0]+1
    
    for e in edges:
        if e.direction == 'R':
            for x in range(e.start[1]+xshift,e.end[1]+xshift+1,1):
                grid[e.start[0]+yshift][x] =1
        if e.direction == 'L':
            for x in range(e.end[1]+xshift,e.start[1]+xshift+1,1):
                grid[e.start[0]+yshift][x] =1
        
        if e.direction == 'D':
            for x in range(e.start[0]+yshift,e.end[0]+yshift+1,1):
                grid[x][e.start[1]+xshift] =1
        
        if e.direction == 'U':
            for x in range(e.end[0]+yshift,e.start[0]+yshift+1,1):
                grid[x][e.start[1]+xshift] =1
    return grid

def fill_grid(grid):
    nR, nC = grid.shape
    
    def give_neighbors(loc):
        dirs = [ (0,1), (0,-1),
                 (1,0), (-1,0)]
        neighs = []
        for c in dirs:
            new_loc = [ i+j for i,j in zip(loc,c)]
            if (new_loc[0]>=0 and new_loc[0] < nR
                and new_loc[1]>=0 and new_loc[1] < nC):
                neighs.append(new_loc)
                
        return neighs
                
    
    search = deque([(0,0)])
    grid[0][0]=2
    
    while len(search) >0:
        cur = search.popleft()
        for n in give_neighbors(cur):
            if grid[n[0]][n[1]] ==0:
                search.append(n)
                grid[n[0]][n[1]] =2
                
    return grid
        
    


In [30]:
def do_part_one(file_path, show=True):
    edges=[]
    with open(file_path,'r') as f:
        cur_loc = (0,0)
        for line in f.readlines():
            line = line.strip()
            d,s,c = line.split()
            edges.append(Edge(cur_loc, d, s, c))
            
            cur_loc = edges[-1].end
            
    grid = create_grid(edges)
    grid = fill_grid(grid)
    
    if show:
        print('\n'.join([''.join([ '#' if x<2 else '.' for x in row]) for row in grid]))

    
    return int((grid<2).sum())

In [31]:
%%time
do_part_one('input_data/test_18.txt')

62.0
.........
.#######.
.#######.
.#######.
...#####.
...#####.
.#######.
.#####...
.#######.
..######.
..######.
.........
CPU times: user 1.46 ms, sys: 1.2 ms, total: 2.66 ms
Wall time: 2.14 ms


62

In [9]:
%%time
do_part_one('input_data/day_18.txt',show=False)

CPU times: user 143 ms, sys: 7.93 ms, total: 150 ms
Wall time: 148 ms


47139

# Part 2

In [10]:
import numpy as np

class EdgeHex:
    def __init__(self, cur_loc, hex_code):
        code = hex_code.replace(')','').replace('(#','')
        steps = int(code[:5], 16)
        dirs = ['R', 'D', 'L','U']
        direction = dirs[int(code[-1])] 
                 
        changes = {
            'U' : np.array([-1,0]),
            'D' : np.array([1,0]),
            'R' : np.array([0,1]),
            'L' : np.array([0,-1]),
        }
        self.start = cur_loc
        self.end = np.array(cur_loc) +(changes[direction] * int(steps))
        
        self.steps = steps
        
        self.direction = direction
        
    def __repr__(self):
        return f'{self.start} -> {self.end}'

In [20]:
def shoelace_algo_trap(edges):
    area=0
    for p1,p2  in zip(edges, edges[1:]+[edges[0]]) :
        area+= (p1.start[0] + p2.start[0])*(p1.start[1] - p2.start[1])
    return area*0.5
            

In [32]:
# Use shoelace formula and Picks Theorem 

def do_part_two(file_path, show=True):
    edges=[]
    with open(file_path,'r') as f:
        cur_loc = (0,0)
        for line in f.readlines():
            line = line.strip()
            d,s,c = line.split()
            edges.append(EdgeHex(cur_loc, c))
            cur_loc = edges[-1].end
    
    shoelace = shoelace_algo_trap(edges)
    pick = sum([ e.steps for e in edges])
            
    return int(shoelace + pick/2 +1)

In [34]:
%%time
do_part_two('input_data/test_18.txt')
           

CPU times: user 718 µs, sys: 590 µs, total: 1.31 ms
Wall time: 837 µs


952408144115

In [35]:
%%time
do_part_two('input_data/day_18.txt')
           

CPU times: user 9.87 ms, sys: 5.64 ms, total: 15.5 ms
Wall time: 11.4 ms


173152345887206